In [1]:
from src import *
import warnings
warnings.filterwarnings("ignore")

## Table of Contents
- [Timeseries Comparison](#comparison)
- [GLM](#weightings)

<a class="anchor" id="comparison"></a>
### Timeseries Comparison 
Comparison between item timeseries and centroid of region timeseries

In [7]:
# 1. Loading Gradient Example
networks = list(index2region.keys())
moviename = 'Sintel'

G_movie, varM = load('./resources/dynamic_grad/group_level414/{}_40_grad.pkl'.format(moviename))

# 2. Loading Matching Emotion Track
full_df  = pd.read_csv('./data/emoFiles/emotion_compile.csv')
emo_df   = full_df[full_df['filename']=='W_{}13.csv'.format(moviename)]
other_df = full_df[full_df['filename']!='W_{}13.csv'.format(moviename)]

In [11]:
select = ['Anger','Guilt','WarmHeartedness', \
             'Disgust','Happiness','Fear','Contempt','Anxiety', \
             'Satisfaction','Shame','Surprise','Love','Sad']

In [15]:
# Grid for each network of correlation with emotions
grad_idx = 0
smfactor = 1
tolshift = 0

n = len(networks)
m = len(select)

pairs = [("Vis", "Default"), ("SomMot", "Default"), 
        ("SomMot", "Vis"), ("DorsAttn", "SalVentAttn"),
        ("DorsAttn", "Default"), ("Limbic", "Default"), ("Cont", "Default")]
nb_metric = 6
scores  = np.zeros((nb_metric, n,m))
nscores = np.zeros((nb_metric, n,m))

# METRIC 1
for idx, pair in enumerate(tqdm(pairs)):
    p1,p2 = pair
    y = np.asarray([networks_distance(G_movie[t], G_movie[t], p1, p2) 
                    for t in range(len(G_movie))])
    z1 = zscore(y)
    for jdx, emotion in enumerate(select):
        concat_other = np.array(other_df[other_df.item == emotion]['score'])
        emo_series = np.array(emo_df[emo_df.item==emotion]['score'])
        smoothened = overlap_add(emo_series, smfactor)
        z2   = zscore(smoothened[:z1.shape[0]])

        _, nscore, corr = moviemix_stat_test(z1, z2, concat_other)
        nscores[0, idx,jdx] = nscore
        scores[0, idx,jdx]  = corr           

# # METRIC 2
# for idx, R in enumerate(tqdm(networks)):
#     y  = network_variance(G_movie, R, grad_idx)
#     z1 = zscore(y)

#     for jdx, emotion in enumerate(select):
#         concat_other = np.array(other_df[other_df.item == emotion]['score'])
#         emo_series = np.array(emo_df[emo_df.item==emotion]['score'])
#         smoothened = overlap_add(emo_series, smfactor)
#         z2   = zscore(smoothened[:z1.shape[0]])

#         _, nscore, corr = moviemix_stat_test(z1, z2, concat_other)
#         nscores[1, idx,jdx] = nscore
#         scores[1, idx,jdx]  = corr        
    
# METRIC 3
for idx, R in enumerate(tqdm(networks)):
    y  = np.asarray([network_volume(G_movie[t], R) 
                    for t in range(len(G_movie))])
    z1 = zscore(y)

    for jdx, emotion in enumerate(select):
        concat_other = np.array(other_df[other_df.item == emotion]['score'])
        emo_series = np.array(emo_df[emo_df.item==emotion]['score'])
        smoothened = overlap_add(emo_series, smfactor)
        z2   = zscore(smoothened[:z1.shape[0]])

        _, nscore, corr = moviemix_stat_test(z1, z2, concat_other)
        nscores[2, idx,jdx] = nscore
        scores[2, idx,jdx]  = corr

# METRIC 4
for idx, R in enumerate(tqdm(networks)-4):
    y  = varM[:,idx]
    z1 = zscore(y)

    for jdx, emotion in enumerate(select):
        concat_other = np.array(other_df[other_df.item == emotion]['score'])
        emo_series = np.array(emo_df[emo_df.item==emotion]['score'])
        smoothened = overlap_add(emo_series, smfactor)
        z2   = zscore(smoothened[:z1.shape[0]])

        _, nscore, corr = moviemix_stat_test(z1, z2, concat_other)
        nscores[3, idx,jdx] = nscore
        scores[3, idx,jdx]  = corr


# METRIC 5 first 7 region of subcortical
for idx, R in enumerate(tqdm(networks)):
    tmpA = G_movie[:, 400+idx][:-1]
    tmpB = G_movie[:, 400+idx][1:]

    y  = np.asarray([ points_distance(tmpA[pidx], tmpB[pidx], pmethod="L2") 
                        for pidx in range(len(tmpA))])
    z1 = zscore(y)

    for jdx, emotion in enumerate(select):
        concat_other = np.array(other_df[other_df.item == emotion]['score'])
        emo_series = np.array(emo_df[emo_df.item==emotion]['score'])
        smoothened = overlap_add(emo_series, smfactor)
        z2   = zscore(smoothened[:z1.shape[0]])

        _, nscore, corr = moviemix_stat_test(z1, z2, concat_other)
        nscores[4, idx,jdx] = nscore
        scores[4, idx,jdx]  = corr

# METRIC 6 second set of 7 region of subcortical
for idx, R in enumerate(tqdm(networks)):
    tmpA = G_movie[:, 407+idx][:-1]
    tmpB = G_movie[:, 407+idx][1:]

    y  = np.asarray([ points_distance(tmpA[pidx], tmpB[pidx], pmethod="L2") 
                        for pidx in range(len(tmpA))])
    z1 = zscore(y)

    for jdx, emotion in enumerate(select):
        concat_other = np.array(other_df[other_df.item == emotion]['score'])
        emo_series = np.array(emo_df[emo_df.item==emotion]['score'])
        smoothened = overlap_add(emo_series, smfactor)
        z2   = zscore(smoothened[:z1.shape[0]])

        _, nscore, corr = moviemix_stat_test(z1, z2, concat_other)
        nscores[5, idx,jdx] = nscore
        scores[5, idx,jdx]  = corr



 29%|██▊       | 2/7 [01:04<02:40, 32.12s/it]


KeyboardInterrupt: 